In [1]:
import os
import pandas as pd 
import numpy as np
import datetime as d
import calendar as c 
from datetime import timedelta

# 数据保证起始站点为NYC站点，可视化系统的数据域为NYC站点
# new header ride_id | rideable_type | started_at | ended_at | start_station_name | start_station_id | end_station_name | end_station_id | start_lat | start_lng | end_lat| end_lng | member_casual
# old header tripduration | starttime | stoptime | start station id | start station name | start station latitude | start station longitude | end station id | end station name | end station latitude | end station longitude | bikeid | usertype | birth year | gender
raw_dirs = ['.\\old_raw\\', '.\\new_raw\\']
require_lists = [[['start station name', 'end station name'], ['start station name', 'start station latitude','start station longitude']],
                 [['start_station_name', 'end_station_name'], ['start_station_name', 'start_lat','start_lng']]]
station_NYC = []

for i in range(2):
    files = os.listdir(raw_dirs[i])
    path_list = [(raw_dirs[i] + f) for f in files if f.endswith('.csv')]

    for p in path_list:
        data = pd.read_csv(p, low_memory=False)
        data = data[data[require_lists[i][0]].notnull().all(1)]
        station_info = data[require_lists[i][1]].drop_duplicates()
        if i == 0:
            station_info.rename(columns={'start station name' : 'start_station_name', 'start station latitude' : 'start_lat', 'start station longitude' : 'start_lng'}, inplace=True) 
        station_NYC.append(station_info)

station_NYC = pd.concat(station_NYC, ignore_index=True)

In [2]:
station_NYC.drop_duplicates(inplace=True)
station_NYC

,start_station_name,start_lat,start_lng
0,Central Park West & W 76 St,40.778968,-73.973747
1,Pershing Square North,40.751873,-73.977706
2,Amsterdam Ave & W 82 St,40.785247,-73.976673
3,1 Ave & E 16 St,40.732219,-73.981656
4,Great Jones St,40.727434,-73.993790
...,...,...,...
37875,W 15 St & 7 Ave,40.780578,-73.985624
37878,Crescent St & 30 Ave,40.770374,-73.928607
37879,Broad St & Bridge St,40.717548,-74.013221
37886,FDR Drive & E 35 St,40.751581,-73.977910


In [3]:
to_list = station_NYC.loc[station_NYC.duplicated(subset=['start_lat','start_lng'],keep=False)].sort_values(['start_lat','start_lng'])
to_list

,start_station_name,start_lat,start_lng
26894,5 Ave & 67 St,40.635679,-74.020005
26903,5 Ave & 66 St,40.635679,-74.020005
386,39 St & 2 Ave - Citi Bike HQ at Industry City,40.655400,-74.010628
10556,39 St & 2 Ave,40.655400,-74.010628
701,2 Ave & 36 St - Citi Bike HQ at Industry City,40.657089,-74.008702
...,...,...,...
20766,E 170 St & Webster Ave_old,40.836103,-73.907199
18577,3 Ave & E 174 St,40.842240,-73.898500
23163,3 Ave & E 174 St - Bathgate Vaccination Site,40.842240,-73.898500
30753,Grand Concourse & E 192 St,40.864480,-73.895220


In [4]:
to_list = to_list.groupby(['start_lat','start_lng'])
to_list

In [5]:
raw_dirs = ['.\\old_raw\\', '.\\new_raw\\']
data_lists = [[['start station name', 'end station name'], ['starttime', 'stoptime', 'start station name', 'start station latitude','start station longitude', 'end station name', 'end station latitude','end station longitude']],
                 [['start_station_name', 'end_station_name'], ['started_at', 'ended_at', 'start_station_name', 'start_lat','start_lng', 'end_station_name', 'end_lat','end_lng']]]
station_data = []

for i in range(2):
    files = os.listdir(raw_dirs[i])
    path_list = [(raw_dirs[i] + f) for f in files if f.endswith('.csv')]

    for p in path_list:
        data = pd.read_csv(p, low_memory=False, usecols=data_lists[i][1])
        data = data[data[data_lists[i][0]].notnull().all(1)]
        if i == 0:
            data.rename(columns={'start station name' : 'start_station_name', 'start station latitude' : 'start_lat', 'start station longitude' : 'start_lng',
                                 'end station name' : 'end_station_name', 'end station latitude' : 'end_lat', 'end station longitude' : 'end_lng', 'starttime' : 'started_at', 'stoptime' : 'ended_at'},inplace=True) 
        station_data.append(data)

In [6]:
station_data = pd.concat(station_data, ignore_index=True)
station_data
    

,started_at,ended_at,start_station_name,start_lat,start_lng,end_station_name,end_lat,end_lng
0,2019-01-01 00:01:47.4010,2019-01-01 00:07:07.5810,Central Park West & W 76 St,40.778968,-73.973747,W 89 St & Columbus Ave,40.788221,-73.970416
1,2019-01-01 00:04:43.7360,2019-01-01 00:10:00.6080,Pershing Square North,40.751873,-73.977706,E 39 St & 2 Ave,40.747804,-73.973442
2,2019-01-01 00:06:03.9970,2019-01-01 00:15:55.4380,Amsterdam Ave & W 82 St,40.785247,-73.976673,E 77 St & 3 Ave,40.773142,-73.958562
3,2019-01-01 00:07:03.5450,2019-01-01 00:52:22.6500,1 Ave & E 16 St,40.732219,-73.981656,W 15 St & 6 Ave,40.738046,-73.996430
4,2019-01-01 00:07:35.9450,2019-01-01 00:12:39.5020,Great Jones St,40.727434,-73.993790,E 20 St & Park Ave,40.738274,-73.987520
...,...,...,...,...,...,...,...,...
57409784,2021-08-31 17:52:14,2021-08-31 18:11:03,Flushing Ave & Vanderbilt Ave,40.697950,-73.970776,6 Ave & 9 St,40.668127,-73.983776
57409785,2021-08-16 18:42:15,2021-08-16 18:59:23,W 147 St & Adam Clayton Powell Blvd,40.822810,-73.937413,W 116 St & Amsterdam Ave,40.806758,-73.960708
57409786,2021-08-09 16:58:49,2021-08-09 17:13:51,W 82 St & Central Park West,40.782750,-73.971370,W 110 St & Amsterdam Ave,40.802692,-73.962950
57409787,2021-08-15 12:24:58,2021-08-15 12:39:29,Grand St & Elizabeth St,40.718822,-73.995960,Greenwich Ave & Charles St,40.735238,-74.000271


In [29]:
drop_index = []
correct_name = []
for i, v in to_list:
    loc_data = station_data.loc[(station_data['start_lat'] == i[0]) & (station_data['start_lng'] == i[1])]
    loc_num = loc_data.shape[0]
    correct_name.clear()
    same_loc = True

    for j in range(v.shape[0]):
        item = v.iloc[j,0]

        item_data = station_data.loc[(station_data['start_station_name'] == item)]
        item_lat = item_data.iloc[0,3]
        item_lng = item_data.iloc[0,4]
        
        if item_lat != i[0] or item_lng != i[1]:
            same_loc = False
        else:
            correct_name.append(item)


    if same_loc:
        print('rename', i, correct_name[0])
        drop_index.extend(v.index[1:])
    elif len(correct_name):
        print('register wrong name', i, correct_name[0])
        drop_index.extend(v.index)
    else:
        print('error data', i, loc_num)
        drop_index.extend(v.index)


rename (40.635679, -74.020005) 5 Ave & 67 St
rename (40.65539977447831, -74.01062786579132) 39 St & 2 Ave - Citi Bike HQ at Industry City
rename (40.65708866668485, -74.00870203971863) 2 Ave & 36 St - Citi Bike HQ at Industry City
register wrong name (40.6711978, -73.97484126) Garfield Pl & 8 Ave
register wrong name (40.6767, -73.969024) Park Pl & Vanderbilt Ave
rename (40.682601, -73.938037) Marcus Garvey Blvd & Macon St
register wrong name (40.6881529, -73.99520919) Congress St & Clinton St
register wrong name (40.68825516598005, -73.99545192718506) 4455.10
rename (40.688489, -73.99116) Boerum Pl\t& Pacific St
error data (40.68848905639242, -73.99116039276123) 5653
error data (40.692, -73.992) 4
rename (40.69757, -73.93098) Willoughby Ave & Myrtle Ave
rename (40.698477, -73.98384) Nassau St\t& Duffield St
rename (40.70684203101325, -73.9544353965357) Division Av & Hooper St
rename (40.70831794366396, -74.00504082441329) Cliff St & Fulton St
register wrong name (40.71146364, -74.00552

In [30]:
m_station_NYC = station_NYC.drop(index=drop_index)
m_station_NYC

,start_station_name,start_lat,start_lng
0,Central Park West & W 76 St,40.778968,-73.973747
1,Pershing Square North,40.751873,-73.977706
2,Amsterdam Ave & W 82 St,40.785247,-73.976673
3,1 Ave & E 16 St,40.732219,-73.981656
4,Great Jones St,40.727434,-73.993790
...,...,...,...
36929,Grand Concourse & E 144 St,40.816870,-73.927985
36960,Vesey St & Church St,40.712220,-74.010472
37561,E 39 St & 2 Ave,40.748033,-73.973828
37794,E 91 St & 2 Ave,40.781152,-73.949630


In [31]:
view_list = m_station_NYC.loc[m_station_NYC.duplicated(subset=['start_lat','start_lng'],keep=False)].sort_values(['start_lat','start_lng'])
view_list

,start_station_name,start_lat,start_lng


In [ ]:
m_station_NYC.to_csv('.\\m_station_NYC.csv')